First one should do:

python serialize_lsst_model.py data/PLASTICC_2018-04-14/LSST_DDF_MODEL01/ --timed

python serialize_lsst_model.py data/PLASTICC_2018-04-14/LSST_DDF_MODEL41/ --timed

python serialize_lsst_model.py data/PLASTICC_2018-04-14/LSST_DDF_MODEL42/ --timed

python serialize_lsst_model.py data/PLASTICC_2018-04-14/LSST_WFD_MODEL01/ --timed

python serialize_lsst_model.py data/PLASTICC_2018-04-14/LSST_WFD_MODEL41/ --timed

python serialize_lsst_model.py data/PLASTICC_2018-04-14/LSST_WFD_MODEL42/ --timed

python serialize_lsst_model.py data/IDEAL_2018-04-14/IDEAL_zALL_MODEL01/ --timed

python serialize_lsst_model.py data/IDEAL_2018-04-14/IDEAL_zALL_MODEL41/ --timed

python serialize_lsst_model.py data/IDEAL_2018-04-14/IDEAL_zALL_MODEL42/ --timed

python serialize_lsst_model.py data/IDEAL_2018-04-14/IDEAL_z02_MODEL01/ --timed

python serialize_lsst_model.py data/IDEAL_2018-04-14/IDEAL_z02_MODEL41/ --timed

python serialize_lsst_model.py data/IDEAL_2018-04-14/IDEAL_z02_MODEL42/ --timed

for each model

and move to pkl/MODEL41-42/

In [1]:
import gzip
import pickle
import pandas as pd
import numpy as np
import os as os
from os import listdir,makedirs,path

LSST_FILTERS = 'ugrizY'


def read_data(filename):
    """Read data from pickled file to a pandas dataframe"""
    with gzip.open(filename, 'rb') as f:
        data = pickle.load(f)

    X = to_dataframe(data)
    y = pd.get_dummies(X.type == 0, prefix='SNIa', drop_first=True)
    X = X.drop(columns=['type'])

    return X, y


def to_dataframe(data):
    """Converts from a python dictionary to a pandas dataframe"""
    for idx in data:
        sn = data[idx]
        for filt in LSST_FILTERS:
            sn['mjd_%s' % filt] = np.array(sn[filt]['mjd'])
            sn['fluxcal_%s' % filt] = np.array(sn[filt]['fluxcal'])
            sn['fluxcalerr_%s' % filt] = np.array(sn[filt]['fluxcalerr'])
            del sn[filt]
        sn.update(sn['header'])
        del sn['header']

    return pd.DataFrame.from_dict(data, orient='index')

Reading the data:

In [2]:
X1, y1 = read_data('LSST_WFD_MODEL41.pkl.gz')
X2, y2 = read_data('LSST_DDF_MODEL41.pkl.gz')
X3, y3 = read_data('IDEAL_z02_MODEL41.pkl.gz')
X4, y4 = read_data('IDEAL_zALL_MODEL41.pkl.gz')
X5, y5 = read_data('LSST_WFD_MODEL42.pkl.gz')
X6, y6 = read_data('LSST_DDF_MODEL42.pkl.gz')
X7, y7 = read_data('IDEAL_z02_MODEL42.pkl.gz')
X8, y8 = read_data('IDEAL_zALL_MODEL42.pkl.gz')
X9, y9 = read_data('LSST_WFD_MODEL01.pkl.gz')
X10, y10 = read_data('LSST_DDF_MODEL01.pkl.gz')
X11, y11 = read_data('IDEAL_z02_MODEL01.pkl.gz')
X12, y12 = read_data('IDEAL_zALL_MODEL01.pkl.gz')

stores all redshifts in a file:

In [3]:
files=['model41_WFD','model41_DDF','model41_ideal_z02','model41_ideal_zALL','model42_WFD','model42_DDF','model42_ideal_z02','model42_ideal_zALL','model01_WFD','model01_DDF','model01_ideal_z02','model01_ideal_zALL']
mat=[X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12]

for i,matr in enumerate(mat):
    file = open(files[i]+'_z.txt','w') 
    for sn in range(len(matr)):
        file.write(str(matr.iloc[sn]['z'])+'\n')
    file.close() 

Prepare SNANA input files:

In [4]:
#write files for snana

for j,matr in enumerate(mat):
    print('Starting '+files[j])
    if not os.path.exists(files[j]):
            os.makedirs(files[j])
    filelist = open(files[j]+'/'+files[j][:7]+'.LIST','w') 
    for sn in range(len(matr)):
        filelist.write(files[j][:7]+'_'+str(matr.iloc[sn]['snid'])+'.txt\n')
        file = open(files[j]+'/'+files[j][:7]+'_'+str(matr.iloc[sn]['snid'])+'.txt','w')     
        file.write('SURVEY:  '+files[j][:7]+'\n')
        file.write('PHOTOMETRY_VERSION: '+files[j][:7]+'\n')
        file.write('SNTYPE: '+files[j][5:7]+'\n')
        file.write('SNID:        '+str(matr.iloc[sn]['snid'])+'\n')
        file.write('IAUC:    UNKNOWN\n')
        file.write('RA:      '+str(matr.iloc[sn]['SIM_RA'])+'  deg\n')
        file.write('DECL:    '+str(matr.iloc[sn]['SIM_DECL'])+'  deg\n')
        file.write('MWEBV:   '+str(matr.iloc[sn]['SIM_MWEBV'])+'  MW E(B-V)\n')
        file.write('REDSHIFT_HELIO:      '+str(matr.iloc[sn]['z'])+' +- 0.000100000 (CMB)\n')
        file.write('REDSHIFT_FINAL:      '+str(matr.iloc[sn]['z'])+' +- 0.000100000 (CMB)\n')
        file.write('FILTERS: ugrizY\n')
        file.write('SEARCH_PEAKMJD: '+str(matr.iloc[sn]['SIM_PEAKMJD'])+'\n')
        file.write('NOBS:           '+str(len(matr.iloc[sn]['mjd_u'])+len(matr.iloc[sn]['mjd_g'])+len(matr.iloc[sn]['mjd_r'])+len(matr.iloc[sn]['mjd_i'])+len(matr.iloc[sn]['mjd_z'])+len(matr.iloc[sn]['mjd_Y']))+'\n')
        file.write('NVAR:   6\n')
        file.write('VARLIST:  MJD  FLT FIELD   FLUXCAL   FLUXCALERR    MAG     MAGERR\n')
        for i in range(len(matr.iloc[sn]['mjd_u'])): 
            file.write('OBS:       '+str(matr.iloc[sn]['mjd_u'][i])+' u WFD '+str(matr.iloc[sn]['fluxcal_u'][i])+' '+str(matr.iloc[sn]['fluxcalerr_u'][i])+' '+str(-2.5*np.log10(matr.iloc[sn]['fluxcal_u'][i])+27.5)+' '+str(2.5*np.log10(1.+(matr.iloc[sn]['fluxcalerr_u'][i]/matr.iloc[sn]['fluxcal_u'][i])))+'\n')
        for i in range(len(matr.iloc[sn]['mjd_g'])): 
            file.write('OBS:       '+str(matr.iloc[sn]['mjd_g'][i])+' g WFD '+str(matr.iloc[sn]['fluxcal_g'][i])+' '+str(matr.iloc[sn]['fluxcalerr_g'][i])+' '+str(-2.5*np.log10(matr.iloc[sn]['fluxcal_g'][i])+27.5)+' '+str(2.5*np.log10(1.+(matr.iloc[sn]['fluxcalerr_g'][i]/matr.iloc[sn]['fluxcal_g'][i])))+'\n')
        for i in range(len(matr.iloc[sn]['mjd_r'])): 
            file.write('OBS:       '+str(matr.iloc[sn]['mjd_r'][i])+' r WFD '+str(matr.iloc[sn]['fluxcal_r'][i])+' '+str(matr.iloc[sn]['fluxcalerr_r'][i])+' '+str(-2.5*np.log10(matr.iloc[sn]['fluxcal_r'][i])+27.5)+' '+str(2.5*np.log10(1.+(matr.iloc[sn]['fluxcalerr_r'][i]/matr.iloc[sn]['fluxcal_r'][i])))+'\n')
        for i in range(len(matr.iloc[sn]['mjd_i'])): 
            file.write('OBS:       '+str(matr.iloc[sn]['mjd_i'][i])+' i WFD '+str(matr.iloc[sn]['fluxcal_i'][i])+' '+str(matr.iloc[sn]['fluxcalerr_i'][i])+' '+str(-2.5*np.log10(matr.iloc[sn]['fluxcal_i'][i])+27.5)+' '+str(2.5*np.log10(1.+(matr.iloc[sn]['fluxcalerr_i'][i]/matr.iloc[sn]['fluxcal_i'][i])))+'\n')
        for i in range(len(matr.iloc[sn]['mjd_z'])): 
            file.write('OBS:       '+str(matr.iloc[sn]['mjd_z'][i])+' z WFD '+str(matr.iloc[sn]['fluxcal_z'][i])+' '+str(matr.iloc[sn]['fluxcalerr_z'][i])+' '+str(-2.5*np.log10(matr.iloc[sn]['fluxcal_z'][i])+27.5)+' '+str(2.5*np.log10(1.+(matr.iloc[sn]['fluxcalerr_z'][i]/matr.iloc[sn]['fluxcal_z'][i])))+'\n')
        for i in range(len(matr.iloc[sn]['mjd_Y'])): 
            file.write('OBS:       '+str(matr.iloc[sn]['mjd_Y'][i])+' Y WFD '+str(matr.iloc[sn]['fluxcal_Y'][i])+' '+str(matr.iloc[sn]['fluxcalerr_Y'][i])+' '+str(-2.5*np.log10(matr.iloc[sn]['fluxcal_Y'][i])+27.5)+' '+str(2.5*np.log10(1.+(matr.iloc[sn]['fluxcalerr_Y'][i]/matr.iloc[sn]['fluxcal_Y'][i])))+'\n')
        file.write('END:')       
        file.close()    
    filelist.close()
    filelist = open(files[j]+'/'+files[j][:7]+'.README','w') 
    filelist.close()
    filelist = open(files[j]+'/'+files[j][:7]+'.IGNORE','w') 
    filelist.close()


Starting model01_WFD


/Users/lluisgalbany/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in log10
/Users/lluisgalbany/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in log10
/Users/lluisgalbany/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in log10
/Users/lluisgalbany/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in log10
/Users/lluisgalbany/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in log10
/Users/lluisgalbany/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in log10


In [ ]:
#import sys,os,math
import numpy as np
from matplotlib import pyplot as plt

SN_Hb = np.linspace(1.,10.,num=100)
SN_Ha = 2.7*SN_Hb # (naively?) assuming constant noise and thermal ratio

# 12+log(O/H) = 8.743 + 0.462*log10(NII/Ha) (N2 calibration by Marino et al. 2013)
OH_min = 8.743 + 0.462*np.log10(1/SN_Ha) # so that S/N>1 for [NII]

# N2 = log10(NII/Ha) = ( 12+log(O/H) - 8.743 ) / 0.462 (from the N2 calibration above)
# 12+log(O/H) = 8.533 - 0.214*( log10(OIII/Hb) - N2 )  (O3N2 calibration by Marino et al. 2013)
# 12+log(O/H) * (1-0.214/0.462) = 8.533 - 0.214*( log10(OIII/Hb) + 8.743/0.462 ) (combining both)
OH_max = ( 8.533 - 0.214*( np.log10(1/SN_Hb) + 8.743/0.462 ) ) / (1-0.214/0.462) # so that S/N>1 for [OIII]

plt.fill_between(SN_Hb, OH_max, 11, facecolor='red', alpha=0.5)
plt.fill_between(SN_Hb, 7, OH_min, facecolor='blue', alpha=0.5)

plt.xlabel(r'(S/N)_H$\beta$')
plt.ylabel(r'12 + log(O/H)')
plt.ylim(8.0,8.8)
plt.text(2.,8.7,r'(S/N)_OIII<1')
plt.text(2.,8.1,r'(S/N)_NII<1')
plt.text(5.,8.4,r'allowed region')

plt.show()